In [ ]:
import pandas as pd
import scipy.signal
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL as pil
import numpy as np
import cv2
import sys
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from tensorflow import keras
import PIL as pil
import multiprocessing as mp
import pickle
from keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class df_holder:
    def __init__(self, df): 
        self.df = df

lead_data = pd.read_pickle("/content/drive/MyDrive/ML_Project_Amo/class_dataset.pkl")

In [ ]:
train_1 = lead_data.loc[:, ['V1', 'V1_df', 'V2', 'V2_df']]
test = lead_data.loc[:, ['V3', 'V3_df']]
a_df = pd.concat([train_1['V1'], train_1['V2']])
b_df = pd.concat([train_1['V1_df'], train_1['V2_df']])

In [ ]:
# create new dataframe with concatenated columns
train = pd.DataFrame({'target': a_df.values, 'df_holder': b_df.values})
train
test = test.rename(columns={'V3': 'target', 'V3_df': 'df_holder'})
test
X_train_, y_train = train['df_holder'].values, train['target'].values
X_test_, y_test = test['df_holder'].values, test['target'].values

In [ ]:
import numpy as np
X_test = np.zeros((202, 5000, 12))
X_train = np.zeros((404, 5000, 12))
print(X_test.shape)
for i in range(len(X_test)):
    X_test[i] = X_test_[i].df[1:5001].values.astype(float)
for i in range(len(X_train)):
    X_train[i] = X_train_[i].df[1:5001].values.astype(float)

(202, 5000, 12)


In [ ]:
print(train['target'].value_counts())

0    301
1    103
Name: target, dtype: int64


In [ ]:
# Separating classes
from sklearn.utils import resample
burntout = train[train['target'].values == 1]
not_burntout = train[train['target'].values == 0]

undersample = resample(burntout, 
                       replace=True, 
                       n_samples=len(not_burntout), #set the number of samples to equal the number of the minority class
                       random_state=42)
# Returning to new training set
train_balanced = pd.concat([not_burntout, undersample])

print(train_balanced['target'].value_counts())

X_train_, y_train = train_balanced['df_holder'].values, train_balanced['target'].values

import numpy as np
X_test = np.zeros((202, 5000, 12))
X_train = np.zeros((404, 5000, 12))
print(X_test.shape)
for i in range(len(X_test)):
    X_test[i] = X_test_[i].df[1:5001].values.astype(float)
for i in range(len(X_train)):
    X_train[i] = X_train_[i].df[1:5001].values.astype(float)

0    301
1    301
Name: target, dtype: int64
(202, 5000, 12)


# don't run from here

In [ ]:
os.mkdir('drive/MyDrive/Spectrograms/train')
for j in range(len(X_train)):
  os.mkdir(f'drive/MyDrive/Spectrograms/train/Participant{j+1}')
  for i in range(12):
    data = pd.DataFrame(X_train[j])
    data = data.iloc[:,i]
    # plt.figure()
    plt.specgram(data)
    # plt.show()
    plt.savefig(f'drive/MyDrive/Spectrograms/train/Participant{j+1}/lead{i+1}.png')

In [ ]:
os.mkdir('drive/MyDrive/Spectrograms/test')
for j in range(len(X_test)):
  os.mkdir(f'drive/MyDrive/Spectrograms/test/Participant{j+1}')
  for i in range(12):
    data = pd.DataFrame(X_train[j])
    data = data.iloc[:,i]
    # plt.figure()
    plt.specgram(data)
    # plt.show()
    plt.savefig(f'drive/MyDrive/Spectrograms/test/Participant{j+1}/lead{i+1}.png')

## run from here

In [ ]:
train_path = os.listdir("/content/drive/MyDrive/Spectrograms/train")
test_path = os.listdir("/content/drive/MyDrive/Spectrograms/test")

In [ ]:
def process_image(path):
  img = pil.Image.open(path)
  img = img.resize((224,224))
  img = img.convert('RGB')
  data = np.asarray(img)
  return data

In [ ]:
train_data = []
for i in train_path:
  print(i)
  participant_path = os.listdir("/content/drive/MyDrive/Spectrograms/train/"+i)
  input_vector = []
  with mp.Pool(processes=8) as pool:
    input_paths = [f"/content/drive/MyDrive/Spectrograms/train/{i}/{j}" for j in participant_path]
    input_vector = pool.map(process_image, input_paths)
  array = np.array(input_vector)
  train_data.append(array)

train_data = np.array(train_data)

Participant1
Participant2
Participant3
Participant4
Participant5
Participant6
Participant7
Participant8
Participant9
Participant10
Participant11
Participant12
Participant13
Participant14
Participant15
Participant16
Participant17
Participant18
Participant19
Participant20
Participant21
Participant22
Participant23
Participant24
Participant25
Participant26
Participant27
Participant28
Participant29
Participant30
Participant31
Participant32
Participant33
Participant34
Participant35
Participant36
Participant37
Participant38
Participant39
Participant40
Participant41
Participant42
Participant43
Participant44
Participant45
Participant46
Participant47
Participant48
Participant49
Participant50
Participant51
Participant52
Participant53
Participant54
Participant55
Participant56
Participant57
Participant58
Participant59
Participant60
Participant61
Participant62
Participant63
Participant64
Participant65
Participant66
Participant67
Participant68
Participant69
Participant70
Participant71
Participant72
P

In [ ]:
test_data = []
for i in test_path:
  print(i)
  participant_path = os.listdir("/content/drive/MyDrive/Spectrograms/test/"+i)
  input_vector = []
  with mp.Pool(processes=8) as pool:
    input_paths = [f"/content/drive/MyDrive/Spectrograms/test/{i}/{j}" for j in participant_path]
    input_vector = pool.map(process_image, input_paths)
  array = np.array(input_vector)
  test_data.append(array)

test_data = np.array(test_data)

Participant1
Participant2
Participant3
Participant4
Participant5
Participant6
Participant7
Participant8
Participant9
Participant10
Participant11
Participant12
Participant13
Participant14
Participant15
Participant16
Participant17
Participant18
Participant19
Participant20
Participant21
Participant22
Participant23
Participant24
Participant25
Participant26
Participant27
Participant28
Participant29
Participant30
Participant31
Participant32
Participant33
Participant34
Participant35
Participant36
Participant37
Participant38
Participant39
Participant40
Participant41
Participant42
Participant43
Participant44
Participant45
Participant46
Participant47
Participant48
Participant49
Participant50
Participant51
Participant52
Participant53
Participant54
Participant55
Participant56
Participant57
Participant58
Participant59
Participant60
Participant61
Participant62
Participant63
Participant64
Participant65
Participant66
Participant67
Participant68
Participant69
Participant70
Participant71
Participant72
P

In [ ]:
file_name_test = "/content/drive/MyDrive/Spectrograms/test_set"
file_name_train = "/content/drive/MyDrive/Spectrograms/train_set"

train_set = train_data

with open(file_name_train, 'wb') as f:
  pickle.dump(train_data,f)

test_set = test_data

with open(file_name_test, 'wb') as f:
  pickle.dump(test_data,f)

In [ ]:
new_y_train = []
for i in range(len(y_train)):
  temp = [y_train[i]]*12
  new_y_train.append(temp)

y_train = np.array(new_y_train)

In [ ]:
len(y_train)
y_train = y_train.flatten()
len(y_train)

4848

In [ ]:
new_y_test = []
for i in range(len(y_test)):
  temp = [y_test[i]]*12
  new_y_test.append(temp)

y_test = np.array(new_y_test)

In [ ]:
with open('/content/drive/MyDrive/Spectrograms/train_set', 'rb') as f:
    train_data = pickle.load(f)

with open('/content/drive/MyDrive/Spectrograms/test_set', 'rb') as f:
    test_data = pickle.load(f)

In [ ]:
X_train = []

for i in range(len(train_data)):
  for j in range(len(train_data[0])):
    X_train.append(train_data[i][j])

len(X_train)
X_train = np.array(X_train)
# X_test, y_test = test_set

In [ ]:
vgg_model = VGG16(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
input_shape = (12, 224, 224, 3)
input_tensor = Input(shape=input_shape)

# Reshape input tensor to have the correct shape
# x = tf.reshape(input_tensor, [-1, input_shape[1], input_shape[2], input_shape[3]])


input_tensor = Input(shape=(224, 224, 3))
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Add your own layers on top of the VGG16 model here
# ...

# model = Model(inputs=input_tensor, outputs=...)


In [ ]:
for layer in vgg_model.layers:
  layer.trainable = False
# Make sure you have frozen the correct layers
for i, layer in enumerate(vgg_model.layers):
    print(i, layer.name, layer.trainable)


0 input_3 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [ ]:
x = vgg_model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
x = Dense(256, activation='relu')(x)
# logits = Dense(12, activation='sigmoid')(x)
x = Dense(1, activation='sigmoid')(x) # Softmax for multiclass
transfer_model = keras.Model(inputs=vgg_model.input, outputs=x)

In [ ]:
learning_rate= 5e-5

transfer_model.compile(loss="binary_crossentropy",optimizer=optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
history = transfer_model.fit(X_train, y_train, batch_size = 12, epochs=30)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
404/404 [==============================] - 34s 60ms/step - loss: 1.2251 - accuracy: 0.6580
Epoch 2/30
404/404 [==============================] - 25s 61ms/step - loss: 0.6225 - accuracy: 0.7158
Epoch 3/30
404/404 [==============================] - 26s 64ms/step - loss: 0.5823 - accuracy: 0.7331
Epoch 4/30
404/404 [==============================] - 26s 64ms/step - loss: 0.5727 - accuracy: 0.7442
Epoch 5/30
404/404 [==============================] - 25s 62ms/step - loss: 0.5554 - accuracy: 0.7496
Epoch 6/30
404/404 [==============================] - 25s 63ms/step - loss: 0.5502 - accuracy: 0.7512
Epoch 7/30
404/404 [==============================] - 26s 64ms/step - loss: 0.5419 - accuracy: 0.7533
Epoch 8/30
404/404 [==============================] - 26s 63ms/step - loss: 0.5384 - accuracy: 0.7572
Epoch 9/30
404/404 [==============================] - 25s 63ms/step - loss: 0.5334 - accuracy: 0.7609
Epoch 10/30
404/404 [==============================] - 26s 63ms/step - loss: 0.530

In [ ]:
X_test = []

for i in range(len(test_data)):
  for j in range(len(test_data[0])):
    X_test.append(test_data[i][j])

len(X_test)
X_test = np.array(X_test)

predictions = transfer_model.predict(X_test)

76/76 [==============================] - 19s 190ms/step


In [ ]:
print(predictions)

[[0.30905515]
 [0.29025945]
 [0.34192652]
 ...
 [0.75726324]
 [0.66712   ]
 [0.53793895]]


In [ ]:
y_pred = []

i = 0
z = 0

while i < len(predictions):
    mean = 0
    while z < 12:
        mean += predictions[i]
        z += 1
        i += 1
    z = 0
    mean = mean/12 
    if mean < 0.5:
        y_pred.append(0)
    else:
        y_pred.append(1)

print(y_pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(len(y_pred))

202


In [ ]:
print(type(y_test))

<class 'numpy.ndarray'>


In [ ]:
print(type(y_pred))

<class 'list'>


In [ ]:
y_pred = np.array(y_pred)

In [ ]:
y_test = np.argmax(y_test, axis=0)
y_pred = np.argmax(y_pred, axis=0)

In [ ]:
print(y_test)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
test = []
for i in range(len(y_test)):
    test.append(y_test[i])

y_test = np.array(test)